In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver

import time
from time import gmtime, strftime

import datetime
import pandas as pd
import os
import numpy as np
import json

now=strftime('%Y-%m-%d',gmtime())
current=pd.to_datetime(str(datetime.datetime.today())[:10])

# zentoto 사이트의 경기 링크 크롤링

In [2]:
driver=webdriver.Chrome("C:\\Users\\PC\\Documents\\chromedriver_win32\\chromedriver")
game_link=[]
import datetime
import numpy as np
for i in range(0,(pd.to_datetime(current)-pd.to_datetime('2018-10-01')).days-1):
    date=pd.to_datetime('2018-10-01')+ datetime.timedelta(days=i)
    url='https://www.zentoto.com/live/'+str(date.year)+'/'+np.asscalar(np.where(date.month<10,'0'+str(date.month),str(date.month)))+'/'+np.asscalar(np.where(date.day<10,'0'+str(date.day),str(date.day)))
    driver.get(url)
    game_link = game_link+[x.get_attribute('href') for x in driver.find_elements_by_css_selector('.list.proto-game > a')]
    game_link = game_link+[x.get_attribute('href') for x in driver.find_elements_by_css_selector('.list.proto-not > a')]
driver.close()

# 각 경기별로 배당흐름 크롤링

In [3]:
def get_odds_flow(link):
    
    home_win_bet = []
    home_draw_bet = []
    home_lose_bet = []

    con = requests.get(link+'#odds')
    soup = BeautifulSoup(con.content, 'lxml')

    infoTable = soup.find("table",{"class":"wikitable sortable"})
    
    a = soup.select('#odds > div > table > tbody > tr > td')
    bet = pd.DataFrame({"win":[], "draw":[], "lose":[]})
    j = 1
    k=0
    for i in soup.select('#odds > div > table > tbody > tr > td > span'):
        if j%3 == 1 :
            win = i.text
            j+=1
        elif j%3 == 2 :
            draw = i.text
            j+=1
        elif j%3 == 0 :
            lose = i.text
            bet.loc[k] = [win, draw, lose]
            j+=1
            k+=1

    j=0
    date_list=[]
    for i in soup.select('#odds > div > table > tbody > tr > td'):
        if j%5==0:
            date_list.append(i.text)
        j+=1

    for var in ['win','draw','lose']:
        bet[var] = bet[var].str.slice(0,4).astype(float)

    bet['date']=date_list
    bet['league_name']=soup.select_one('span.fts16').text
    bet['game_date']=soup.select_one('p.fts12').text
    bet['result']=soup.select_one('div.game-text.game-status-finish').text
    teams=[x.text for x in soup.select('p.tac.name-kor')]
    bet['home_team_kor']=teams[0]
    bet['away_team_kor']=teams[1]

    teams=[x.text for x in soup.select('p.tac.name-eng')]
    bet['home_team_eng']=teams[0]
    bet['away_team_eng']=teams[1]
    return bet

In [4]:
agg={}
not_work=[]
for link in game_link:
    try:
        bet=get_odds_flow(link)
        bet['game_link']=link
        agg[link]=bet
    except:
        pass

In [5]:
os.chdir('C:\\Users\\PC\\Documents\\odds')
#문서 안에 odds 폴더를 만드시고 실행해주세요

final=pd.concat(agg,ignore_index=True)
final=final.rename(columns={'date':'odds_change_date'})
final.to_csv("final_odds.csv",index=False)